In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the data

In [54]:
dataset = pd.read_csv('https://raw.githubusercontent.com/revanthmadasu/LifestyleWellbeing/master/Wellbeing_and_lifestyle_data_Kaggle.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Converting categorial data into numerical

Understanding which objects to convert

In [48]:
object_dataset = dataset.select_dtypes(include=['object']).copy()
object_dataset.head()

,DAILY_STRESS,AGE,GENDER
0,2,36 to 50,Female
1,3,36 to 50,Female
2,3,36 to 50,Female
3,3,51 or more,Female
4,1,51 or more,Female


Converting Timestamp using label encoding.
"Finding and replacing" method would take forever.
"One-hot encoder" method would creat a lot more unnecessary columns


In [55]:
dataset['Timestamp'] = dataset['Timestamp'].astype('category')
dataset['Timestamp'] = dataset['Timestamp'].cat.codes
dataset.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,5607,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,5607,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,5607,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,5607,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,5607,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


Using "Finding and replacing" method, since there are values only from 0 to 5, with a arbitrary value of 1/1/2000 in row 10007

In [56]:
dataset['DAILY_STRESS'] = dataset['DAILY_STRESS'].replace(['0','1','2','3','4','5'], [0,1,2,3,4,5])
dataset.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,5607,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,5607,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,5607,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,5607,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,5607,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


Using get_dummies method for 'AGE' & 'GENDER' category

In [57]:
pd.get_dummies(dataset, columns = ['AGE', 'GENDER']).head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,WORK_LIFE_BALANCE_SCORE,AGE_21 to 35,AGE_36 to 50,AGE_51 or more,AGE_Less than 20,GENDER_Female,GENDER_Male
0,5607,3,2,2,5,0,5,2,0,1,...,4,0,5,609.5,0,1,0,0,1,0
1,5607,2,3,4,3,8,10,5,2,2,...,3,2,6,655.6,0,1,0,0,1,0
2,5607,2,3,3,4,4,10,3,2,2,...,4,8,3,631.6,0,1,0,0,1,0
3,5607,3,3,10,3,10,7,2,5,2,...,5,2,0,622.7,0,0,1,0,1,0
4,5607,5,1,3,3,10,4,2,4,2,...,8,1,5,663.9,0,0,1,0,1,0


In [ ]:
print(X[0,:])

['7/7/15' 3 '2' 2 5 0 5 2 0 1 6 4 5 0 7 5 5 1 4 0 5 '36 to 50' 'Female']


In [ ]:
print(X)

[['7/7/15' 3 '2' ... 5 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 6 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 3 '36 to 50' 'Female']
 ...
 ['3/14/21 8:35' 4 '3' ... 10 '21 to 35' 'Male']
 ['3/14/21 8:43' 1 '1' ... 8 '21 to 35' 'Female']
 ['3/14/21 9:03' 5 '4' ... 4 '21 to 35' 'Female']]


In [ ]:
print(y)

[609.5 655.6 631.6 ... 716.6 682.  651.4]


Handling missing data

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer.fit(X[:, 1:3])
# X[:, 1:3] = imputer.transform(X[:, 1:3])